In [9]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

#target_url = "https://bsr.sudrf.ru/bigs/portal.html#%7B%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:%7B%22queryRequests%22:%5B%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22case_document_result_date%5C%22,%5C%22operator%5C%22:%5C%22B%5C%22,%5C%22query%5C%22:%5C%222018-01-01T00:00:00%5C%22,%5C%22sQuery%5C%22:%5C%222018-12-31T00:00:00%5C%22%7D,%7B%5C%22name%5C%22:%5C%22case_document_category_article_cat%5C%22,%5C%22operator%5C%22:%5C%22SEW%5C%22,%5C%22query%5C%22:%5C%22%D0%A1%D1%82%D0%B0%D1%82%D1%8C%D1%8F%20229.1%20%D0%A7%D0%B0%D1%81%D1%82%D1%8C%203%5C%22%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22common%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES%22%7D%5D%7D,%22sorts%22:%5B%7B%22field%22:%22score%22,%22order%22:%22desc%22%7D%5D,%22simpleSearchFieldsBundle%22:%22default%22,%22filterGroups%22:%5B%7B%22filterQueries%22:%5B%7B%22field%22:%22case_court_type%22,%22query%22:%22%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9,%20%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D1%81%D0%BA%D0%BE%D0%B9,%20%D0%BC%D0%B5%D0%B6%D1%80%D0%B0%D0%B9%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9%20%D1%81%D1%83%D0%B4%22,%22not%22:false%7D%5D,%22groupMode%22:%22OR%22,%22not%22:false%7D%5D%7D"
target_url = "https://bsr.sudrf.ru/bigs/portal.html#%7B%22start%22:0,%22rows%22:1000,%22uid%22:%22d7c55781-7685-44b9-9149-115cd9df3d01%22,%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:%7B%22queryRequests%22:%5B%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22case_document_result_date%5C%22,%5C%22operator%5C%22:%5C%22B%5C%22,%5C%22query%5C%22:%5C%222018-01-01T00:00:00%5C%22,%5C%22sQuery%5C%22:%5C%222018-12-31T00:00:00%5C%22%7D,%7B%5C%22name%5C%22:%5C%22case_document_category_article_cat%5C%22,%5C%22operator%5C%22:%5C%22SEW%5C%22,%5C%22query%5C%22:%5C%22%D0%A1%D1%82%D0%B0%D1%82%D1%8C%D1%8F%20229.1%20%D0%A7%D0%B0%D1%81%D1%82%D1%8C%203%5C%22%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22common%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES%22%7D%5D%7D,%22sorts%22:%5B%7B%22field%22:%22score%22,%22order%22:%22desc%22%7D%5D,%22simpleSearchFieldsBundle%22:%22default%22,%22filterGroups%22:%5B%7B%22filterQueries%22:%5B%7B%22field%22:%22case_court_type%22,%22query%22:%22%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9,%20%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D1%81%D0%BA%D0%BE%D0%B9,%20%D0%BC%D0%B5%D0%B6%D1%80%D0%B0%D0%B9%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9%20%D1%81%D1%83%D0%B4%22,%22not%22:false%7D%5D,%22groupMode%22:%22OR%22,%22not%22:false%7D%5D,%22facet%22:%7B%22field%22:%5B%22type%22%5D%7D,%22facetLimit%22:21,%22additionalFields%22:%5B%22court_document_documentype1%22,%22court_case_entry_date%22,%22court_case_result_date%22,%22court_subject_rf%22,%22court_name_court%22,%22court_document_law_article%22,%22court_case_result%22,%22case_user_document_type%22,%22case_user_doc_entry_date%22,%22case_user_doc_result_date%22,%22case_doc_subject_rf%22,%22case_user_doc_court%22,%22case_document_category_article%22,%22case_user_doc_result%22,%22case_user_entry_date%22,%22m_case_user_type%22,%22m_case_user_sub_type%22,%22ora_main_law_article%22%5D,%22hlFragSize%22:1000,%22groupLimit%22:3,%22woBoost%22:false%7D"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from web_parser import Page

p = Page(target_url)

In [13]:
p.current_page_case_urls[0]

'https://bsr.sudrf.ru/bigs/showDocument.html#id=0d9bd36643ac3b90550ac542d06543dc&shard=%D0%92%D1%81%D0%B5%20%D0%B4%D0%B5%D0%BB%D0%B0%20(%D0%BD%D0%BE%D0%B2%D0%BE%D0%B5)&from=p&r={%22start%22:0,%22rows%22:1000,%22uid%22:%22d7c55781-7685-44b9-9149-115cd9df3d01%22,%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:{%22queryRequests%22:[{%22type%22:%22Q%22,%22request%22:%22{\\%22mode\\%22:\\%22EXTENDED\\%22,\\%22typeRequests\\%22:[{\\%22fieldRequests\\%22:[{\\%22name\\%22:\\%22case_document_result_date\\%22,\\%22operator\\%22:\\%22B\\%22,\\%22query\\%22:\\%222018-01-01T00:00:00\\%22,\\%22sQuery\\%22:\\%222018-12-31T00:00:00\\%22},{\\%22name\\%22:\\%22case_document_category_article_cat\\%22,\\%22operator\\%22:\\%22SEW\\%22,\\%22query\\%22:\\%22%D0%A1%D1%82%D0%B0%D1%82%D1%8C%D1%8F%20229.1%20%D0%A7%D0%B0%D1%81%D1%82%D1%8C%203\\%22}],\\%22mode\\%22:\\%22AND\\%22,\\%22name\\%22:\\%22common\\%22,\\%22typesMode\\%22:\\%22AND\\%22}]}%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES

In [11]:
len(p.current_page_case_ids)

71

In [14]:
import traceback
import requests
import pandas as pd

for uid, url in zip(p.current_page_case_ids, p.current_page_case_urls):
    data = {"request":{"id": uid}}
    try:
        resp = requests.post("https://bsr.sudrf.ru/bigs/showDocument.action",
                                 json=data,
                                 headers={"Content-Type": "application/json"})
        fields = resp.json()['document']["fields"]
        df = pd.DataFrame(fields)
        df["case_url"] = url
        case_title = df[df["comment"]=="Название"]["value"].values[0]
        try:
            datetime_str = df[df["comment"]=="Дата решения"]["dateValue"].values[0]
        except IndexError:
            datetime_str = "no_datetime"
        fname = fname = "" + case_title + " " + datetime_str + ".csv"
        fname = fname.replace(" ", "_").replace("/", "_")
        fpath = "~/PycharmProjects/prigovor_parser/cases/" + fname
        print(fpath)
        df.to_csv(fpath, index=None, header=True)
    except BaseException as e:
        print(traceback.format_exc())

~/PycharmProjects/prigovor_parser/cases/Уголовное_дело_1-53_2018_2018-03-26T04:00:00.csv
~/PycharmProjects/prigovor_parser/cases/Уголовное_дело_1-17_2018_2018-02-26T04:00:00.csv
~/PycharmProjects/prigovor_parser/cases/Уголовное_дело_1-37_2018_2018-05-08T04:00:00.csv
~/PycharmProjects/prigovor_parser/cases/Уголовное_дело_1-51_2018_2018-04-12T04:00:00.csv
~/PycharmProjects/prigovor_parser/cases/Уголовное_дело_1-14_2018_(1-139_2017;)_no_datetime.csv
~/PycharmProjects/prigovor_parser/cases/Уголовное_дело_1-19_2018_(1-146_2017;)_2018-02-07T01:00:00.csv
~/PycharmProjects/prigovor_parser/cases/Уголовное_дело_1-30_2018_no_datetime.csv
~/PycharmProjects/prigovor_parser/cases/Уголовное_дело_1-27_2018_2018-08-27T04:00:00.csv
~/PycharmProjects/prigovor_parser/cases/Уголовное_дело_1-30_2018_(1-602_2017;)_2018-05-11T04:00:00.csv
~/PycharmProjects/prigovor_parser/cases/Уголовное_дело_1-14_2018_2018-04-13T04:00:00.csv
~/PycharmProjects/prigovor_parser/cases/Уголовное_дело_1-24_2018_2018-03-02T04:00:00

In [142]:
import pandas as pd

pd.set_option('display.max_rows', 500)
df = pd.DataFrame(case_fields).dropna(axis='columns', how="all").drop(columns=["linkField", "linkName", "linkValue", "markup", "highlighting"])

In [ ]:
df